# 1 Dependencies import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scripts.quickml.reader import read_csv
from scripts.quickml.plot import dataframe_distribution_overview, feature_filling, feature_distribution_univar, feature_trend_bivar, feature_distribution_bivar, feature_distribution_multivar
from scripts.quickml.compare import index_lth
from scripts.utils import fill_infos

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)

np.random.seed(294697)

***
# 2 Data import

For performance reasons, i choose to write a function that allow me to "chunk" the dataset and only take a percentage. I use a method for taking randomly the rows in each chunk, the result is that the output dataset have the same shape than if it was the original one.

In [ ]:
original_dataset = read_csv(pd, np, "./datas/en.openfoodfacts.org.products.csv", 5, delimiter="\t", chunk_size=10000, nrows=None)

In [ ]:
rows_nb, cols_nb = original_dataset.shape
original_dataset.shape

In [ ]:
original_dataset.head()

In [ ]:
data = original_dataset.copy()

***
# 3 Cleaning

In [ ]:
dataframe_distribution_overview(data, figsize=(60, 2))

### Shape comparison with original dataset

Original dataset  
![dataset_100_percent](images/global_overview_100p.png)

In [ ]:
features = []

***
## 3.1 General informations group

### 3.1.1 Overview

In [ ]:
general_infos_features = data.loc[:, "code":"quantity"].columns
general_infos_features

In [ ]:
general_infos = data.loc[:, general_infos_features]

In [ ]:
dataframe_distribution_overview(general_infos)

In [ ]:
general_infos.head()

In [ ]:
general_infos.info()

***
### 3.1.2 Usability

#### 3.1.2.1 Usability: Quantity

In [ ]:
feature_filling(general_infos, "quantity", "Quantity")

In [ ]:
general_infos[general_infos.quantity.notna()].quantity.head(10)

In [ ]:
feature_distribution_univar(general_infos, "quantity", "Quantity", (20, 3))

The feature "quantity" is highly non homogeneous and will be very difficult to use. We will not keep it.

In [ ]:
general_infos.drop(columns=["quantity"], inplace=True)

#### 3.1.2.2 Usability: abbreviated_product_name

In [ ]:
feature_filling(general_infos, "abbreviated_product_name", "Abbreviated product name")

In [ ]:
general_infos[general_infos["abbreviated_product_name"].notna()].loc[:, ["abbreviated_product_name"]].head()

The fill percentage of 'abbreviated_product_name" is very low, we can rid of it.

In [ ]:
general_infos.drop(columns=["abbreviated_product_name"], inplace=True)

#### 3.1.2.3 Usability: generic_name

In [ ]:
feature_filling(general_infos, "generic_name", "Generic name")

In [ ]:
general_infos.drop(columns=["generic_name"], inplace=True)

***
### 3.1.3 Redundancy

***
### 3.1.4 Conclusion

In [ ]:
dataframe_distribution_overview(general_infos)

#### 3.1.4.1 Original

In [ ]:
dataframe_distribution_overview(data[general_infos_features])

In [ ]:
plt.figure(figsize=(10,3))

barplot = sns.barplot(x=general_infos_features, y=data[general_infos_features].count())

plt.title("Number of values per column", size=20)
plt.xticks(rotation=45, size=16, ha="right")
plt.yticks(size=16)
plt.ylabel("Number values", size=16)
plt.show()

***
## 3.2 Tags group

### 3.2.1 Overview

In [ ]:
tags_features = data.loc[:, "packaging":"countries_en"].columns
tags_features

In [ ]:
tags = data.loc[:, tags_features]

In [ ]:
dataframe_distribution_overview(tags)

In [ ]:
tags.head()

In [ ]:
tags.iloc[:, :13].info()

In [ ]:
tags.iloc[:, 13:].info()

***
### 3.2.2 Usability

#### 3.2.2.1 Usability: packaging_text

In [ ]:
feature_filling(tags, "packaging_text", "Packaging text")

In [ ]:
tags.drop(columns=["packaging_text"], inplace=True)

***
#### 3.2.2.2 Usability: cities

In [ ]:
feature_filling(tags, "cities", "Cities")

In [ ]:
tags.drop(columns=["cities"], inplace=True)

***
### 3.2.3 Redundancy

#### 3.2.3.1 Redundancy: packaging, packaging_tags

In [ ]:
tags.loc[:, ["packaging", "packaging_tags"]].loc[tags.packaging.notna() & tags.packaging_tags.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Packaging", tags["packaging"].astype("category").cat.codes),
    ("Packagin tags", tags["packaging_tags"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["packaging_code"] = tags["packaging"].astype("category").cat.codes
temp["packaging_tags_code"] = tags["packaging_tags"].astype("category").cat.codes

temp.corr()

We can see that each time one of the features "packaging" or "packaging_tags" is filled, the other is also filled.
We will keep the feature "packaging" because it transport more information, plus, it's easier to get rid of the "en:", "fr:" than "en-", because in "packaging_tags" each separator is a "-" and not in "packaging".

In [ ]:
tags.drop(columns=["packaging_tags"], inplace=True)

***
#### 3.2.3.2 Redundancy: brands, brands_tags

In [ ]:
tags.iloc[:20].loc[:, ["brands", "brands_tags"]]

In [ ]:
feature_distribution_multivar([
    ("Brands", tags["brands"].astype("category").cat.codes),
    ("Brands tags", tags["brands_tags"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["brands_code"] = tags["brands"].astype("category").cat.codes
temp["brands_tags_code"] = tags["brands_tags"].astype("category").cat.codes

temp.corr()

The features "brands" and "brands_tags" are almost equally filled. We will keep "brands" because this feature carry most information (spaces, capital letters...)

In [ ]:
tags.drop(columns=["brands_tags"], inplace=True)

***
#### 3.2.3.3 Redundancy: categories, categories_tags, categories_en

In [ ]:
tags.loc[:, ["categories", "categories_tags", "categories_en"]].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Categories", tags["categories"].astype("category").cat.codes),
    ("Categories tags", tags["categories_tags"].astype("category").cat.codes),
    ("Categories en", tags["categories_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["categories_code"] = tags["categories"].astype("category").cat.codes
temp["categories_tags_code"] = tags["categories_tags"].astype("category").cat.codes
temp["categories_en_code"] = tags["categories_en"].astype("category").cat.codes

temp.corr()

We will keep "categories_en".

In [ ]:
tags.drop(columns=["categories", "categories_tags"], inplace=True)

***
#### 3.2.3.4 Redundancy: origins, origins_tags, origins_en

In [ ]:
tags.loc[:, ["origins", "origins_tags", "origins_en"]].loc[tags.origins.notna() & tags.origins_tags.notna() & tags.origins_en.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Origins", tags["origins"].astype("category").cat.codes),
    ("Origins tags", tags["origins_tags"].astype("category").cat.codes),
    ("Origins en", tags["origins_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["origins_code"] = tags["origins"].astype("category").cat.codes
temp["origins_tags_code"] = tags["origins_tags"].astype("category").cat.codes
temp["origins_en_code"] = tags["origins_en"].astype("category").cat.codes

temp.corr()

We will keep the feature "origins"

In [ ]:
tags.drop(columns=["origins_tags", "origins_en"], inplace=True)

***
#### 3.2.3.5 Redundancy: manufacturing_places, manufacturing_places_tags

In [ ]:
tags.loc[:, ["manufacturing_places", "manufacturing_places_tags"]].loc[tags.manufacturing_places.notna() & tags.manufacturing_places_tags.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Manufacturing places", tags["manufacturing_places"].astype("category").cat.codes),
    ("Manufacturing places tags", tags["manufacturing_places_tags"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["manufacturing_places_code"] = tags["manufacturing_places"].astype("category").cat.codes
temp["manufacturing_places_tags_code"] = tags["manufacturing_places_tags"].astype("category").cat.codes

temp.corr()

We will keep the feature "manufacturing_places"

In [ ]:
tags.drop(columns=["manufacturing_places_tags"], inplace=True)

***
#### 3.2.3.6 labels, labels_tags, labels_en

In [ ]:
tags.loc[:, ["labels", "labels_tags", "labels_en"]].loc[tags.labels.notna() & tags.labels_en.notna() & tags.labels_en.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Labels", tags["labels"].astype("category").cat.codes),
    ("Labels tags", tags["labels_tags"].astype("category").cat.codes),
    ("Labels en", tags["labels_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["labels_code"] = tags["labels"].astype("category").cat.codes
temp["labels_tags_code"] = tags["labels_tags"].astype("category").cat.codes
temp["labels_en_code"] = tags["labels_en"].astype("category").cat.codes

temp.corr()

We will keep "labels_en"

In [ ]:
tags.drop(columns=["labels", "labels_tags"], inplace=True)

***
#### 3.2.3.7 emb_codes, emb_codes_tags

In [ ]:
tags.loc[:, ["emb_codes", "emb_codes_tags"]].loc[tags.emb_codes.notna() & tags.emb_codes_tags.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Emb codes", tags["emb_codes"].astype("category").cat.codes),
    ("Emb codes tags", tags["emb_codes_tags"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["emb_codes_code"] = tags["emb_codes"].astype("category").cat.codes
temp["emb_codes_tags_code"] = tags["emb_codes_tags"].astype("category").cat.codes

temp.corr()

We will keep "emb_codes".

In [ ]:
tags.drop(columns=["emb_codes_tags"], inplace=True)

***
#### 3.2.3.8 countries, countries_tags, countries_en

In [ ]:
tags.iloc[:20,:].loc[:,["countries", "countries_tags", "countries_en"]]

In [ ]:
feature_distribution_multivar([
    ("countries", tags["countries"].astype("category").cat.codes),
    ("countries tags", tags["countries_tags"].astype("category").cat.codes),
    ("countries en", tags["countries_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["countries_code"] = tags["countries"].astype("category").cat.codes
temp["countries_tags_code"] = tags["countries_tags"].astype("category").cat.codes
temp["countries_en_code"] = tags["countries_en"].astype("category").cat.codes

temp.corr()

In [ ]:
tags.drop(columns=["countries", "countries_tags"], inplace=True)

***
### 3.2.4 Conclusion

In [ ]:
tags.describe()

In [ ]:
tags.info()

In [ ]:
dataframe_distribution_overview(tags)

#### 3.2.4.1 Original

In [ ]:
dataframe_distribution_overview(data[tags_features])

***
## 3.3 Ingredients group

### 3.3.1 Overview

In [ ]:
ingredients_features = data.loc[:, "ingredients_text":"traces_tags"].columns
ingredients_features

In [ ]:
ingredients = data.loc[:, ingredients_features]

In [ ]:
dataframe_distribution_overview(ingredients)

In [ ]:
ingredients.info()

In [ ]:
ingredients.head()

***
### 3.3.2 Usability

#### 3.3.2.1 Usability: allergens_en

In [ ]:
feature_filling(ingredients, "allergens_en", "Allergens")

In [ ]:
ingredients.drop(columns=["allergens_en"], inplace=True)

***
### 3.3.3 Redundancy

#### 3.3.3.1 Redundancy: traces, traces_tags

In [ ]:
ingredients.loc[:, ["traces", "traces_tags"]].loc[ingredients.traces.notna() & ingredients.traces_tags.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("traces", ingredients["traces"].astype("category").cat.codes),
    ("traces tags", ingredients["traces_tags"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["traces_code"] = ingredients["traces"].astype("category").cat.codes
temp["traces_tags_code"] = ingredients["traces_tags"].astype("category").cat.codes

temp.corr()

There is, at first sight, no better feature. We will keep "traces"

In [ ]:
ingredients.drop(columns=["traces_tags"], inplace=True)

***
### 3.3.4 Conclusion

In [ ]:
ingredients.describe()

In [ ]:
ingredients.info()

In [ ]:
dataframe_distribution_overview(ingredients)

#### 3.3.4.1 Original

In [ ]:
dataframe_distribution_overview(data[ingredients_features])

***
## 3.4 Misc datas group

#### 3.4.1 Overview

In [ ]:
misc_features = data.loc[:,"serving_size":"image_nutrition_small_url"].columns
misc_features

In [ ]:
misc = data.loc[:, misc_features]

In [ ]:
dataframe_distribution_overview(misc)

In [ ]:
misc.iloc[:, :16].info()

In [ ]:
misc.iloc[:, 16:].info()

In [ ]:
misc.head(1)

***
### 3.4.2 Usability

These features are competely empty and then will be no needed:
- no_nutriments
- additives
- ingredients_from_palm_oil
- ingredients_that_may_be_from_palm_oil

These features are almost empty, we will get rid of them too:
- ingredients_from_palm_oil_tags
- ingredients_that_may_be_from_palm_oil_tags

In [ ]:
feature_filling(misc, "no_nutriments", "No nutriments")

In [ ]:
feature_filling(misc, "additives", "Additives")

In [ ]:
feature_filling(misc, "ingredients_from_palm_oil", "Ingredients from palm oil")

In [ ]:
feature_filling(misc, "ingredients_that_may_be_from_palm_oil", "Ingredients that may be from palm oil")

In [ ]:
feature_filling(misc, "ingredients_from_palm_oil_tags", "Ingredients from palm oil tags")

In [ ]:
feature_filling(misc, "ingredients_that_may_be_from_palm_oil_tags", "Ingredients that may be from palm oil tags")

In [ ]:
misc.drop(columns=["no_nutriments", "additives", "ingredients_from_palm_oil", "ingredients_that_may_be_from_palm_oil", "ingredients_from_palm_oil_tags", "ingredients_that_may_be_from_palm_oil_tags"], inplace=True)

***
### 3.4.3 Redundancy

#### 3.4.3.1 Redundancy: serving_siez, serving_quantity

In [ ]:
misc.loc[:, ["serving_size", "serving_quantity"]].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Serging size", misc["serving_size"].astype("category").cat.codes),
    ("Serging size tags", misc["serving_quantity"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["serving_size_code"] = misc["serving_size"].astype("category").cat.codes
temp["serving_quantity_code"] = misc["serving_quantity"].astype("category").cat.codes

temp.corr()

It appears that the feature "serving_size" is non homogeneous in term of units. The feature "serving_quantity" on the other hand, seems to be only in the same unit (g).
We will keep "serving_quantity"

In [ ]:
misc.drop(columns=["serving_size"], inplace=True)

***
#### 3.4.3.2 Redundancy: additives_tags, additives_en

In [ ]:
misc.loc[:, ["additives_n", "additives_tags", "additives_en"]].iloc[:10]

In [ ]:
feature_distribution_multivar([
    ("Additives count", misc["additives_n"].astype("category").cat.codes),
    ("Additives tags", misc["additives_tags"].astype("category").cat.codes),
    ("Additives en", misc["additives_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["additives_n_code"] = misc["additives_n"].astype("category").cat.codes
temp["additives_tags_code"] = misc["additives_tags"].astype("category").cat.codes
temp["additives_en_code"] = misc["additives_en"].astype("category").cat.codes

temp.corr()

The feature "additives_en" contains more informations and seems to be easier to parse compare to "additives_tags". Regarding "additives_n" it's the number of additives, there is no direct redundancy for that feature, we can keep it.
We will keep "additives_en" and "additives_n".

In [ ]:
misc.drop(columns=["additives_tags"], inplace=True)

***
#### 3.4.3.3 Redundancy: nutriscore_score, nutriscore_grade

In [ ]:
misc.loc[:, ["nutriscore_score", "nutriscore_grade"]].loc[misc.nutriscore_score.notna() & misc.nutriscore_grade.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Nutriscore score", misc["nutriscore_score"].astype("category").cat.codes),
    ("Nutriscore grade", misc["nutriscore_grade"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["nutriscore_score_code"] = misc["nutriscore_score"].astype("category").cat.codes
temp["nutriscore_grade_code"] = misc["nutriscore_grade"].astype("category").cat.codes

temp.corr()

We will keep both fields as one is quantitative and other is qualitative.

***
#### 3.4.3.4 Redundancy: pnns_groups_1, pnns_groups_2

In [ ]:
misc.loc[:, ["pnns_groups_1", "pnns_groups_2"]].loc[misc.pnns_groups_1.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Pnns groups 1", misc["pnns_groups_1"].astype("category").cat.codes),
    ("Pnns groups 2", misc["pnns_groups_2"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["pnns_groups_1_code"] = misc["pnns_groups_1"].astype("category").cat.codes
temp["pnns_groups_2_code"] = misc["pnns_groups_2"].astype("category").cat.codes

temp.corr()

Both features don't contains same value, we will keep them.

***
#### 3.4.3.5 Redundancy: states, states_tags, states_en

In [ ]:
misc.loc[:, ["states", "states_tags", "states_en"]].iloc[:10]

In [ ]:
feature_distribution_multivar([
    ("States", misc["states"].astype("category").cat.codes),
    ("States tags", misc["states_tags"].astype("category").cat.codes),
    ("States en", misc["states_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["states_code"] = misc["states"].astype("category").cat.codes
temp["states_tags_code"] = misc["states_tags"].astype("category").cat.codes
temp["states_en_code"] = misc["states_en"].astype("category").cat.codes

temp.corr()

Seems to be all useless. Let's see how many rows doesn't contains 'to-be-completed' or 'To be completed' in those features.

These features contains no usefull informations, we will get rid of them.

In [ ]:
misc.drop(columns=["states", "states_tags", "states_en"], inplace=True)

***
#### 3.4.3.6 Redundancy: ecoscore_score_fr, ecoscore_grade_fr

In [ ]:
misc.loc[:, ["ecoscore_score_fr", "ecoscore_grade_fr"]].loc[misc.ecoscore_score_fr.notna() & misc.ecoscore_grade_fr.notna()].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("Ecoscore score fr", misc["ecoscore_score_fr"].astype("category").cat.codes),
    ("States tags", misc["ecoscore_grade_fr"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["ecoscore_score_fr_code"] = misc["ecoscore_score_fr"].astype("category").cat.codes
temp["ecoscore_grade_fr_code"] = misc["ecoscore_grade_fr"].astype("category").cat.codes

temp.corr()

As the nutriscore, both ecoscore are usefull, we will keep both.

***
#### 3.4.3.7 Redundancy: main_category, main_category_en

In [ ]:
misc.loc[:, ["main_category", "main_category_en"]].iloc[:20]

In [ ]:
feature_distribution_multivar([
    ("main_category", misc["main_category"].astype("category").cat.codes),
    ("main_category en", misc["main_category_en"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["main_category_code"] = misc["main_category"].astype("category").cat.codes
temp["main_category_en_code"] = misc["main_category_en"].astype("category").cat.codes

temp.corr()

We will keep "main_category_en"

In [ ]:
misc.drop(columns=["main_category"], inplace=True)

***
### 3.4.4 Conclusion

In [ ]:
misc.info()

In [ ]:
misc.describe()

In [ ]:
misc.loc[:, :"main_category_en"].describe(include=[object])

In [ ]:
dataframe_distribution_overview(misc)

#### 3.4.4.1 Original

In [ ]:
dataframe_distribution_overview(data[misc_features])

***
## 3.5 Nutrition facts group

### 3.5.1 Overview

In [ ]:
nutr_features = data.loc[:, "energy-kj_100g":"carnitine_100g"].columns
nutr_features

In [ ]:
nutr = data.loc[:, nutr_features]

In [ ]:
dataframe_distribution_overview(nutr, figsize=(30, 2))

In [ ]:
nutr.iloc[:, :10].info()

In [ ]:
nutr.iloc[:, 10:20].info()

In [ ]:
nutr.iloc[:, 20:30].info()

In [ ]:
nutr.iloc[:, 30:40].info()

In [ ]:
nutr.iloc[:, 40:50].info()

In [ ]:
nutr.iloc[:, 50:60].info()

In [ ]:
nutr.iloc[:, 60:70].info()

In [ ]:
nutr.iloc[:, 70:80].info()

In [ ]:
nutr.iloc[:, 80:90].info()

In [ ]:
nutr.iloc[:, 90:100].info()

In [ ]:
nutr.iloc[:, 100:110].info()

In [ ]:
nutr.iloc[:, 110:120].info()

***
### 3.5.2 Usability

All features that have less than 1% fill percentage will be dropped.

In [ ]:
index_lesser_than_percentage = index_lth(pd, np, nutr, 5)
index_lesser_than_percentage

In [ ]:
nutr.drop(columns=index_lesser_than_percentage, inplace=True)

***
### 3.5.3 Redundancy

#### 3.5.3.1 Redundancy: nutrition-score-fr_100g

nutrition-score-fr_100g is redundant with "nutriscore_score" and "nutriscore_grade". It will be dropped.

In [ ]:
feature_distribution_multivar([
    ("nutriscore score", data["nutriscore_score"].astype("category").cat.codes),
    ("nutriscore grade", data["nutriscore_grade"].astype("category").cat.codes),
    ("nutriscore fr 100g", nutr["nutrition-score-fr_100g"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
temp["nutriscore_score_code"] = data["nutriscore_score"].astype("category").cat.codes
temp["nutriscore_grade_code"] = data["nutriscore_grade"].astype("category").cat.codes
temp["nutrition-score-fr_100g_code"] = nutr["nutrition-score-fr_100g"].astype("category").cat.codes

temp.corr()

In [ ]:
nutr.drop(columns=["nutrition-score-fr_100g"], inplace=True)

***
#### 3.5.3.2 Redundancy: energy-kj_100g, energy-kcal_100g, energy_100g

In [ ]:
# feature_distribution_multivar([
#     ("Energy kj 100g", nutr["energy-kj_100g"].astype("category").cat.codes),
#     ("Energy Kcal 100g", nutr["energy-kcal_100g"].astype("category").cat.codes),
#     ("Energy 100g", nutr["energy_100g"].astype("category").cat.codes)
# ])

feature_distribution_multivar([
    ("Energy Kcal 100g", nutr["energy-kcal_100g"].astype("category").cat.codes),
    ("Energy 100g", nutr["energy_100g"].astype("category").cat.codes)
])

In [ ]:
temp = pd.DataFrame()
# temp["energy-kj_100g_code"] = nutr["energy-kj_100g"].astype("category").cat.codes
temp["energy-kcal_100g_code"] = nutr["energy-kcal_100g"].astype("category").cat.codes
temp["energy_100g_code"] = nutr["energy_100g"].astype("category").cat.codes

# temp.loc[:, ["energy-kj_100g_code", "energy-kcal_100g_code", "energy_100g_code"]].corr()
temp.loc[:, ["energy-kcal_100g_code", "energy_100g_code"]].corr()

The "energy-kj_100g" is much emptier than the two other energy features and bring no additional infos.
The feature "energy-kcal_100g" has explicit unit and is the most filled.

We will keep only "energy-kcal_100g"

In [ ]:
# nutr.drop(columns=["energy-kj_100g", "energy_100g"], inplace=True)
nutr.drop(columns=["energy_100g"], inplace=True)

***
### 3.5.4 Conclusion

In [ ]:
nutr.info()

In [ ]:
nutr.describe()

In [ ]:
dataframe_distribution_overview(nutr)

#### 3.5.4.1 Original

In [ ]:
dataframe_distribution_overview(data[nutr_features], figsize=(30, 2))

***
## 3.6 Results

In [ ]:
features = general_infos.join([tags, ingredients, misc, nutr])

In [ ]:
features.shape

In [ ]:
dataframe_distribution_overview(features, figsize=(30, 2))

***
# 4 Saving

In [ ]:
features.shape

In [ ]:
features.to_csv("datas/sample.csv", sep=";")

***
# ? Nutriscore

## Nutriscore feature analysis

In [ ]:
nutriscore_features = [
    "fruits-vegetables-nuts_100g",
    "proteins_100g",
    "fiber_100g",
    "sugars_100g",
    "salt_100g",
    "saturated-fat_100g",
    "energy-kcal_100g"
]

In [ ]:
nutriscore_data = data.loc[:, ["nutriscore_score", "nutriscore_grade", "nutrition-score-fr_100g"]]
nutriscore_data.head(10)

In [ ]:
na = nutriscore_data.any(axis="columns")
print("at least one value: ", na.sum())
print("nutriscore_grade recorded: ", data.nutriscore_grade.notna().sum())
print("all rows: ", na.count())

We can see that these threee features "nutriscore_score", "nutriscore_grade" and "nutrition-fr_100G" are redundant. Each time there is at least one of them recorded, the two others are also filled.

We can then get ride of "nutrition-fr_100g" and keep the two others.

In [ ]:
features += ["nutriscore_score", "nutriscore_grade"]

Too few entries have nutriscore cell filled, around 33%.
Could be a way to calculate it manually from other fields ?

## Nutriscore needed data

Is there a way to calculate Nutriscore from the others information ?

For that, we need to have the P and N ingredients of the nutriscore formula.

P (positive) ingredients:
- fruits, vegetable, nuts
- proteins
- fiber

N (negative) ingredients:
- sugar
- salt
- saturated fatty acid
- energy (kcal)

In [ ]:
p = [
    "fruit",
    "vegetable",
    "nuts",
    "olive",
    "fiber"
]

In [ ]:
n = [
    "sugar",
    "sodium",
    "saturated fatr"
]

## Search for needed fields/features

In [ ]:
results = []
for ingredient in p + n:
    for column in data.columns:
        if ingredient in column:
            results.append(column)
print(len(results))
results

## Nutriscore's features content analysis

They are the fields needed for the nutriscore formulas, but:
- "fruits-vegetables-nuts_100g": completely empty
- "fiber": too fews records, but could simply be due to the fact not all product contains fiber

## Where are the fruits, vegetables and nuts ?

In [ ]:
pd.set_option('display.max_colwidth', None)

ingredient_text_data = data[data.ingredients_text.notna()].ingredients_text
ingredient_text_data.head(5)

In [ ]:
def search(row, col: str, match: list):
    for m in match:
        if m in row[col]:
            print("row[col]: ", row[col])
            # return row[col].index

data[data.ingredients_text.notna()].iloc[:10].apply(lambda row: search(row, "ingredients_text", p), axis=1)

Even if we succed to demonstrate the concentration in fruits/evetables/nuts of certain products, we still can't calculate the nutriscore because we don't have the proportion of those elements per 100g.

### Conclusion
There is now way to calculate the nutriscore.